# LangWatch Batch Evaluation Cookbook

## Step 1: Define our LLM pipeline

Let's create a simple RAG pipeline using LangChain, guaranteeing that we can get the output and the retrieved documents used during generation.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain.prompts import ChatPromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.documents import Document


loader = WebBaseLoader("https://docs.langwatch.ai")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retrieved_documents = []

# Wrap the FAISS retriever so that we can capture which documents were used to generate the response
@tool
def langwatch_search(
    query: str
) -> list[Document]:
    """"Search for information about LangWatch. For any questions about LangWatch, use this tool if you didn't already"""

    global retrieved_documents
    retrieved_documents = retriever.get_relevant_documents(query)
    return retrieved_documents

tools = [langwatch_search]
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that only reply in short tweet-like responses, use tools only once.\n\n{agent_scratchpad}",
        ),
        ("human", "{question}"),
    ]
)
agent = create_tool_calling_agent(model, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=False)  # type: ignore

output = executor.invoke({"question": "What is LangWatch?"})["output"]

print("")
print("retrieved_documents:", [ d.page_content for d in retrieved_documents])
print("output:", output)

USER_AGENT environment variable not set, consider setting it to identify your requests.
/var/folders/rp/9_s_f3kd1ssb089myww_p9zw0000gn/T/ipykernel_23993/189265682.py:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_documents = retriever.get_relevant_documents(query)



retrieved_documents: ['Introduction - LangWatchLangWatch home pageSearch...SupportDashboardlangwatch/langwatchlangwatch/langwatchSearch...NavigationIntroductionDocumentationLangEvalsOpen DashboardGitHub RepoIntroductionSelf-HostingOptimization StudioOverviewLLM NodesDatasetsEvaluatingOptimizingMonitoring IntegrationOverviewPythonTypeScriptOpenTelemetryLangflow IntegrationFlowise IntegrationREST API IntegrationRAG Context TrackingConceptsCookbooksEvaluationsEvaluationsCustom Evaluator IntegrationGuardrailsOverviewSetting Up GuardrailsUser EventsOverviewEventsDSPy VisualizationQuickstartCustom Optimizer TrackingRAG VisualizationMore FeaturesTriggersAnnotationsDatasetsEmbedded AnalyticsAPI EndpointsTracesAnnotationsTroubleshooting and SupportIntroductionWelcome to LangWatch, the all-in-one open-source LLMops platform.\nLangWatch allows you to track, monitor, guardrail and evaluate your LLMs apps for measuring quality and alert on issues.', 'LangWatch allows you to track, monitor, guardra

## Step 2: Run the Batch Evaluation Experiment

Now we can use the dataset we have from LangWatch to run a batch evaluation experiment through our LLM pipeline, to see the results and tweak it for optimizations.

In [2]:
from dotenv import load_dotenv

load_dotenv()

from langwatch.batch_evaluation import BatchEvaluation, DatasetEntry


def callback(entry: DatasetEntry):
    output = executor.invoke({"question": entry["input"]})["output"]

    import random
    if random.random() < 0.3:
        raise Exception("Random Error")

    return {"output": output, "contexts": [d.page_content for d in retrieved_documents]}

# Instantiate the BatchEvaluation object
evaluation = BatchEvaluation(
    experiment="LangWatch RAG Experiment",
    dataset="langwatch-rag",
    evaluations=["openai-moderation","faithfulness"],
    callback=callback,
)

# Run the evaluation
results = evaluation.run()
results.df

Starting batch evaluation...
Follow the results at: http://localhost:5560/inbox-narrator/experiments/langwatch-rag-experiment


100%|██████████| 12/12 [00:24<00:00,  2.06s/it]


Batch evaluation done!


,openai-moderation,faithfulness
0,skipped,skipped
1,skipped,skipped
2,skipped,skipped
3,skipped,skipped
4,skipped,skipped
5,skipped,skipped
6,skipped,skipped
7,True,1.0
8,True,0.666667
9,True,0.0
